In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
dataset = pd.read_csv("Social_Network_Ads.csv")
dataset


,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [2]:
datanomin = pd.get_dummies(dataset, drop_first = True, dtype = int)
datanomin

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [3]:
datanomin=datanomin.drop('User ID', axis=1)
datanomin['Purchased'].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [4]:
datanomin.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [5]:
indep = datanomin[[ 'Age', 'EstimatedSalary', 'Gender_Male']]
dep = datanomin[['Purchased']]

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 0.3, random_state =0)

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
param_grid = {'solver':['newton-cg', 'lbfgs', 'liblinear', 'saga','newton-cholesky','sag'],
             'penalty':['l1', 'l2', 'elasticnet', None]}
grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
55 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1194, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
 

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l1', 'l2', 'elasticnet', None],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga',
                                    'newton-cholesky', 'sag']},
             scoring='f1_weighted', verbose=3)

In [13]:
re=grid.cv_results_
print(re)

{'mean_fit_time': array([4.06188965e-03, 2.04167366e-03, 1.27774239e-02, 6.27848148e-02,
       2.08606720e-03, 1.81508064e-03, 1.44148636e-01, 2.06424904e-01,
       4.26120758e-02, 1.23228073e-02, 2.02471304e-01, 7.05642700e-03,
       0.00000000e+00, 2.02412605e-03, 1.21116638e-05, 2.07366943e-03,
       2.01997757e-03, 1.74231529e-03, 8.99098396e-02, 3.00454140e-02,
       1.66187286e-03, 1.41673088e-02, 1.01129055e-02, 1.23211861e-02]), 'std_fit_time': array([4.97562322e-03, 4.08334732e-03, 4.58652174e-03, 7.29083378e-03,
       3.10458320e-03, 3.63016129e-03, 3.60222717e-02, 1.10746084e-01,
       4.08515232e-02, 4.48732353e-03, 1.57676238e-01, 6.05162053e-03,
       0.00000000e+00, 4.04825211e-03, 2.42233276e-05, 4.14733887e-03,
       4.03995514e-03, 3.48463058e-03, 4.95595071e-02, 5.82839651e-03,
       3.32374573e-03, 4.97969441e-03, 7.39798060e-05, 3.92727596e-03]), 'mean_score_time': array([0.        , 0.        , 0.01611915, 0.03574862, 0.        ,
       0.        , 0.013

In [15]:
grid_predictions = grid.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix (y_test, grid_predictions)
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)
print(cm)
print(clf_report)

[[74  5]
 [ 8 33]]
              precision    recall  f1-score   support

           0       0.90      0.94      0.92        79
           1       0.87      0.80      0.84        41

    accuracy                           0.89       120
   macro avg       0.89      0.87      0.88       120
weighted avg       0.89      0.89      0.89       120



In [16]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("Best Params {}:".format(grid.best_params_),f1_macro)

Best Params {'penalty': 'l2', 'solver': 'newton-cg'}: 0.8906190214115365


In [17]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9484408768138315